<a href="https://colab.research.google.com/github/natdebandi/hate_speech_ar/blob/main/3_GPT_finetuning_bin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP final - reconocimiento de discursos discriminatorios en Twitter
## (3 )Finetuning de GPT 4 mini para clasificación binaria

**Natalia Dedandi**



La documentación de la API se encuentra aqui:

https://platform.openai.com/docs/api-reference/introduction


Guia para finetuning:

https://medium.com/@garethcull/fine-tuning-ai-models-a-practical-guide-for-beginners-dc313b2e0f76

In [8]:
pip install openai

In [9]:
!pip install datasets seaborn


In [10]:
import json
from openai import OpenAI
import os
from google.colab import userdata
#seteo la KEY
os.environ['OPENAI_API_KEY'] = userdata.get('openIA_key')




In [11]:
#creo el cliente OPENAI con mi usuario y proyecto
client = OpenAI(
  organization='org-1uHjwiaB3OlPzoxfVzhqOSzs',
  project='proj_2fII7izwVGgYaERNNKhhMx4l',
)

Levanto los datos de entrenamiento

In [55]:
from datasets import load_dataset
import pandas as pd

ds1 = load_dataset("piuba-bigdata/contextualized_hate_speech")


Tengo que transformar el DS de train en un conjunto de mensajes para pasarle a GPT para finetuning.

In [56]:
# prompt: Create a dataframe from ds1 with id text and hateful cols

df_train = pd.DataFrame(ds1['train'])[['id', 'text', 'HATEFUL']]
df_test = pd.DataFrame(ds1['test'])[['id', 'text', 'HATEFUL']]


In [57]:
# prompt: create a datfram from ds1 for train
df_train[0:5]

,id,text,HATEFUL
0,343726,@usuario Uno menos,1
1,384799,@usuario #QueSeVayanTodos @usuario @usuario @u...,0
2,399435,"@usuario Te falta negociar con Cuba, mamerto.",0
3,403171,@usuario Se prepara para las milicias armadas,0
4,394186,@usuario Seguimos alimentando vagos,1


In [58]:
df_train['hateful_label'] = df_train['HATEFUL'].apply(lambda x: "hateful" if x==1  else "not hateful")
df_test['hateful_label'] = df_test['HATEFUL'].apply(lambda x: "hateful" if x==1  else "not hateful")
df_test[0:5]

,id,text,HATEFUL,hateful_label
0,397703,@usuario Ojalá se funda y cague de hambre,0,not hateful
1,397704,@usuario Faltó que se enganchó al hijo de sofo...,1,hateful
2,397705,"@usuario y bue, que le vaya a llorar a Cristin...",0,not hateful
3,397706,@usuario @usuario Jodete. Macri te hizo conoce...,0,not hateful
4,397707,"@usuario Que se joda, es su gobierno el que el...",0,not hateful


Creo una lista BATCH con el formato de mensajes que debo enviar a GPT para finetunning

Creo primero el archivo train y luego test

In [60]:


  # Convert the dataframe into a list of dictionaries with custom_id and messages
  batch = [
      {'messages': [
                  {'role': 'system', 'content': "You are trained to analyze and detect the sentiment of the given text. Only hateful or not hateful is allowed. If the text is empty or is not clear answer not hateful"},
                  {'role': 'user', 'content': row['text']},
                  {'role': 'assistant', 'content': row['hateful_label']}
                  ]
       }
      for _, row in df_train.iterrows()
  ]


In [62]:
batch[0]

{'messages': [{'role': 'system',
   'content': 'You are trained to analyze and detect the sentiment of the given text. Only hateful or not hateful is allowed. If the text is empty or is not clear answer not hateful'},
  {'role': 'user', 'content': '@usuario Uno menos'},
  {'role': 'assistant', 'content': 'hateful'}]}

In [63]:
# Save the batch to a .jsonl file
with open('batch_gpt_bin_train.jsonl', 'w') as f:
    for entry in batch:
        json.dump(entry, f)
        f.write('\n')

print("Batch file created successfully in .jsonl format!")

Batch file created successfully in .jsonl format!


In [64]:
  # Convert the dataframe into a list of dictionaries with custom_id and messages
  batch_test = [
      {'messages': [
                  {'role': 'system', 'content': "You are trained to analyze and detect the sentiment of the given text. Only hateful or not hateful is allowed. If the text is empty or is not clear answer not hateful"},
                  {'role': 'user', 'content': row['text']},
                  {'role': 'assistant', 'content': row['hateful_label']}
                  ]
       }
      for _, row in df_test.iterrows()
  ]

In [65]:
# Save the batch to a .jsonl file
with open('batch_gpt_bin_test.jsonl', 'w') as f:
    for entry in batch_test:
        json.dump(entry, f)
        f.write('\n')

print("Batch file created successfully in .jsonl format!")

Batch file created successfully in .jsonl format!


Subo ambos archivos a GPT para luego hacer el finetuning

In [66]:
from openai import OpenAI
client = OpenAI()

client.files.create(
  file=open("batch_gpt_bin_train.jsonl", "rb"),
  purpose="fine-tune"
)

FileObject(id='file-Mt9HMlQtIwmiVA8R0OF4aINw', bytes=14301977, created_at=1722722551, filename='batch_gpt_bin_train.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [67]:
from openai import OpenAI
client = OpenAI()

client.files.create(
  file=open("batch_gpt_bin_test.jsonl", "rb"),
  purpose="fine-tune"
)

FileObject(id='file-bBd2qAB8YmmZUUrnkXsVO9wF', bytes=4433834, created_at=1722722554, filename='batch_gpt_bin_test.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [68]:
client.fine_tuning.jobs.create(
  training_file="file-Mt9HMlQtIwmiVA8R0OF4aINw",
  validation_file="file-bBd2qAB8YmmZUUrnkXsVO9wF",
  model="gpt-3.5-turbo-0125",
  hyperparameters={
    "n_epochs":3
  },
  suffix="hate_speech_v2"
)

FineTuningJob(id='ftjob-RI7JuIvpL27w3XMLAIQWTNaw', created_at=1722722606, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-1uHjwiaB3OlPzoxfVzhqOSzs', result_files=[], seed=112634201, status='validating_files', trained_tokens=None, training_file='file-Mt9HMlQtIwmiVA8R0OF4aINw', validation_file='file-bBd2qAB8YmmZUUrnkXsVO9wF', estimated_finish=None, integrations=[], user_provided_suffix='hate_speech_v2')

In [74]:
fine_tuning_job = client.fine_tuning.jobs.retrieve(
  fine_tuning_job_id="ftjob-RI7JuIvpL27w3XMLAIQWTNaw"
)

print(fine_tuning_job)


FineTuningJob(id='ftjob-RI7JuIvpL27w3XMLAIQWTNaw', created_at=1722722606, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=72, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-1uHjwiaB3OlPzoxfVzhqOSzs', result_files=[], seed=112634201, status='validating_files', trained_tokens=None, training_file='file-Mt9HMlQtIwmiVA8R0OF4aINw', validation_file='file-bBd2qAB8YmmZUUrnkXsVO9wF', estimated_finish=None, integrations=[], user_provided_suffix='hate_speech_v2')


In [75]:
fine_tuning_job.status

'validating_files'

In [51]:
#client.fine_tuning.jobs.cancel("ftjob-3hLvVaaM5T51D19lXitFUC5Q")

FineTuningJob(id='ftjob-3hLvVaaM5T51D19lXitFUC5Q', created_at=1722719196, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=5, batch_size=121, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-1uHjwiaB3OlPzoxfVzhqOSzs', result_files=[], seed=1064557800, status='cancelled', trained_tokens=None, training_file='file-I6srp0hpoIQllyxQRQ4U24iB', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix='hate_speech_v2')